In [ ]:
!docker version

In [ ]:
!mkdir flaskwebapp
!mkdir flaskwebapp/code
!mkdir flaskwebapp/etc

In [ ]:
!cp /poc/AZ/Kate/returnsModel_2.1-8.tar.gz flaskwebapp/code/
!cp /poc/AZ/Kate/OR_IR6_2017-11-28_modelling.RData flaskwebapp/code/

In [ ]:
import os
from os import path
import json 

image_name = "thbeh/collections"
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'Dockerfile')

In [ ]:
%%writefile flaskwebapp/packages.R
#install.packages(c("randomForest"), 
#                   repos = "http://cran.us.r-project.org", dependencies = TRUE)

## install.packages("/poc/AZ/Kate/returnsModel_2.1-6.tar.gz", repos = NULL, type = "source")


In [ ]:
%%writefile flaskwebapp/code/scoreDrill.R

#suppressMessages(library(psych))
#suppressMessages(library(jsonlite))
#suppressMessages(library(sergeant))
suppressMessages(library(returnsModel))

options(warn = -1)

#* @post /predict
runModel <- function(nr) {
    RunModellingAndScoringTest(model="OR", returnName="IR6", projectStage="TEST", nrows=nr)
}

#* @post /refreshmodel
refreshmodel <- function(model="OR", returnName="IR6", projectStage="TEST", nrows=1000){
    
    output <- RefreshModel(model=model, returnName=returnName, projectStage=projectStage, nrows=nrows)
    AssignOutput(output, env= environment())
  
    runDate <- format(Sys.Date(), "%Y-%m-%d")
    imageName <- paste0("/code/", model,"_", returnName, "_", runDate, "_modelling.RData") 
    save(list=names(output), file=imageName, envir=e)
    imageName
}


#* @post /testload
testload <- function(){
    e <<- .GlobalEnv
    load("/code/OR_IR6_2017-11-28_modelling.RData", envir=e)
    print(e$covYearsBack)
}


#* @post /doscoring
doscoring <- function(model="OR", returnName="IR6", modelRunDate="2017-11-28", projectStage="TEST", nrows=1000, 
              irdNumbersForScoring=""){
    e <<- .GlobalEnv
    imageName <- paste0(model, "_", returnName, "_", modelRunDate, "_modelling.RData")
    load(paste0("/code/", imageName), envir=e)
    
    e$model <- model
    e$returnName <- returnName
    e$modelRunDate <- modelRunDate
    e$projectStage <- projectStage
    e$nrows <- nrows
    e$irdNumbersForScoring <- irdNumbersForScoring
    
    DoScoring(model=e$model, returnName=e$returnName, modelRunDate=e$modelRunDate, projectStage=e$projectStage, 
              nrows=e$nrows, irdNumbersForScoring=e$irdNumbersForScoring, e=e)
    
}


In [ ]:
%%writefile flaskwebapp/etc/supervisord.conf
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true               ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:plumber]
command=/usr/bin/Rscript -e "pr <- plumber::plumb('/code/scoreDrill.R'); pr$run(host='0.0.0.0', port=8000)"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

In [ ]:
%%writefile flaskwebapp/Dockerfile
FROM thbeh/r-base:3.4.2

RUN apt-get update && apt-get install -y supervisor libxml2-dev

COPY packages.R /tmp/
COPY scoreDrill.R /tmp/

RUN mkdir /code
ADD code /code
ADD etc /etc

RUN Rscript /tmp/packages.R
RUN R CMD INSTALL /code/returnsModel_2.1-8.tar.gz

EXPOSE 8000
CMD ["supervisord","-c", "/etc/supervisord.conf"]


In [ ]:
! ls -l flaskwebapp/code

In [ ]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

In [6]:
!docker rm refreshmodel

Error response from daemon: You cannot remove a running container 4ee8753aef2ede23b6fab85bd12a4e392fd0e8d831ef696aa5cefb33847d8015. Stop the container before attempting removal or use -f


In [ ]:
!docker run -it -e MAPR_CLUSTER=maprlab -e MAPR_CLDB_HOSTS=mapr-1,mapr-2,mapr-3 -e MAPR_CONTAINER_USER=mapr -e MAPR_CONTAINER_UID=5000 -e MAPR_CONTAINER_GID=5000 -e MAPR_CONTAINER_GROUP=mapr -e MAPR_MOUNT_POINT=/mapr --device /dev/fuse --security-opt apparmor:unconfined --name refreshmodel -p 6001:8000 $image_name 

In [5]:
!docker rmi thbeh/collections

Untagged: thbeh/collections:latest
Deleted: sha256:cbbedadf9eda464445064f4ca23dcbeef5a83ac45398baba060dc8904de6636d
Deleted: sha256:68f172009f63aaf588d4fa2b4f918c117670c515d5901da6b18b9e38a9228a73
Deleted: sha256:b7691e2d752337cc155b3a9fb9892db0862dff7c64e75ef2133cabf4decd2799
Deleted: sha256:3c0f8cff3f9da669a8f9a20c1ec3bd6ad319d3c12a2099b6cba83a7fe8d29309
Deleted: sha256:e0778f0ed8e9c63d67e91d26b47b9f767b886d3850fe7a351404a6dcd5efdad3
Deleted: sha256:a7ef82dec81a87a9b7c3a4f99b25e32e2d8714f81bf6c2bb203ecccdea1478c9
Deleted: sha256:1880f7800dc7e04c841700d2f3edb9284df1e693e16d900a0701607556b10bad
Deleted: sha256:117ca0ed8c2152f66db11bac7bd18f7aaf72e95751326a8e88930751d075686b
Deleted: sha256:c5b301c763a4ee1a10c6137f85278689abbd2f063cadf0bb8addd189dc57928b
Deleted: sha256:1ae2fb8080fc10a90928a8f0933bf93cc19921288f8979fcffbd5626c8b08870
Deleted: sha256:72531f767d28aa5b691bb2c795c0923a3efee7bb968c97611795355b9689411b
Deleted: sha256:3321a3390a9858f588ea47301801f150a939f0451e2e716deca8e98277